In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
from sinkhorn_knopp import sinkhorn_knopp as skp
sk = skp.SinkhornKnopp()

class FAQ():
    '''
    Fast Approximate QAP Algorithm (FAQ)
    The FAQ algorithm solves the Quadratic Assignment Problem (QAP) finding an alignment of the
    vertices of two graphs which minimizes the number of induced edge disagreements
    
    
    '''
    def FAQ(A,B):
        '''
        Computes the optimal permutation for QAP using Frank-Wolff optimization
        
        Paramters
        ---------
        A : 2d-array, square
            A square adjacency matrix
            
        B : 2d-array, square
            A square adjacency matrix
            
        Returns
        -------
        perm : array_like
            An array of the optimal permutations as a result of optimization
            Array is length n, where n is the number of vertices in the network
        '''
        n = A.shape[0]  #number of vertices in graphs
        P = Pinit(n)
        At = np.transpose(A)  # A transpose
        Bt = np.transpose(B)  # B transpose
    
        #OPTIMIZATION WHILE LOOP BEGINS
        for i in range(30):
            delfp = A@P@Bt+At@P@B  # computing the gradient of f(P) = -tr(APB^tP^t)
            rows, cols = linear_sum_assignment(-delfp) # run hungarian algorithm on gradient(f(P))
            Q = np.zeros((n,n))  
            Q[rows,cols] = 1   # initialize search direction matrix Q
            def f(x):  #computing the original optimization function
                return np.trace(At@(x*P+(1-x)*Q)@B@np.transpose(x*P+(1-x)*Q))
            alpha = minimize_scalar(f, bounds=(0,1), method='bounded').x #computing the step size
            P = alpha*P + (1-alpha)*Q  # Update P
        
        #end of FW optimization loop
        row, perm = linear_sum_assignment(-P) # run hungarian algorithm on Pfinal
        return perm
        
    def Permutation(arr):
        '''
        Converts a list of permutations to a permutation matrix
        
        Parameters
        ----------
        arr: array_like
            length n array, which is some rearrangement of [0,n)
            
        Returns
        -------
        P : 2d-array
            nxn permutation matrix associated with input
        '''
        n = len(arr) #number of elements in input array
        row = np.array(range(n))
        P = np.zeros((n,n)) #initiate a nxn matrix of zeros
        P[row,arr] = 1 # set indices of permutation to 1
        return P
    
    def Pinit(num): #function that returns a random doubly stochastic starting matrix, with number of vertices as input
        '''
        Creates a random doubly stochastic starting matrix for the purpose of multiple restart testing
        
        Paramters
        ---------
        num : int
            specificies the dimensionality of the output, a num x num matrix
        
        P : 2d_array
            num x num matrix created from the equation (J+K)/2, where J is the doubly stochastic barrycenter and K is some
            random doubly stochastic matrix.   
        '''
        n = num
        sk = skp.SinkhornKnopp() 
        K = np.random.rand(n,n) #generate a nxn matrix where each entry is a random integer [0,1]
        for i in range(10): #perform 10 iterations of Sinkhorn balancing
            K = sk.fit(K)
        J = np.ones((n,n))/float(n) # initialize J, a doubly stochastic barycenter
        P = (K+J)/2
        return P
    
    def ofv(A,B,P): #function to calculate objective function value based on permutation matrix
    '''
    Computes the objective function value of the QAP
    
    Parameters 
    ---------
    A : 2d-array
        A square adjacency matrix
        
    B : 2d-array
        A square adjacency matrix
        
    P : 2d-array
        A square permutation matrix
        
    Returns
    -------
    i : float
        objective function value, trace(APB^tP^t)
    '''
        i = np.trace(np.transpose(A)@P@B@np.transpose(P)) #objective function value
        return i
    
    def test(A,B,num): #testing function where input is number of iterations
        '''
        test performance of FAQ algorithm against multiple random restarts
        
        Parameters
        ----------
        A : 2d-array
            A square adjacency matrix
            
        B : 2d-array
            A square adjacency matrix
            
        num : int
            number of times FAQ is run
        
        Returns 
        -------
        mini : float
            the minimum objective function value resulting from the num FAQ runs
            
        '''
        oofv = np.zeros(num) # initiate array of zeros of length num
        for i in range(num): # loop through num times
            Pi = Permutation(FAQ(A,B)) #run FAQ on input matrices
            oofv[i] = ofv(A,B,Pi) #store objective function value
        mini = min(oofv) #initialize value to store minimum of all ofvs from FAQ run
        return mini


In [ ]:
: